In [1]:
# Read SAS file
import pandas as pd

In [2]:
# NHANES 2007-2008 vaccination Immunization
nhanes78_Vaccination = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/IMQ_E.XPT", format = "xport")

In [3]:
# NHANES 2007-2008 core antibody surface antigen
nhanes78_BG = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/HEPBD_E.XPT", format = "xport" )

In [4]:
# NHANES 2007-2008 surface antibody
nhanes78_Antibody = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/HEPB_S_E.XPT", format = "xport")

In [5]:
nhanes78_Vaccination = nhanes78_Vaccination[['SEQN','IMQ020']]

In [6]:
nhanes78_BG = nhanes78_BG[['SEQN','LBXHBC','LBDHBG']]

In [7]:
nhanes78_Antibody

,SEQN,LBXHBS
0,41475.0,2.0
1,41476.0,1.0
2,41477.0,2.0
3,41479.0,1.0
4,41480.0,NaN
...,...,...
9008,51619.0,2.0
9009,51620.0,2.0
9010,51621.0,1.0
9011,51622.0,2.0


In [8]:
nhanes78_No_vaccination = nhanes78_Vaccination[(nhanes78_Vaccination['IMQ020']==3.0) | (nhanes78_Vaccination['IMQ020']==7.0) | (nhanes78_Vaccination['IMQ020']==9.0)]

In [9]:
nhanes78_No_vaccination

,SEQN,IMQ020
0,41475.0,3.0
2,41477.0,3.0
7,41482.0,3.0
8,41483.0,3.0
10,41485.0,9.0
...,...,...
10142,51617.0,9.0
10143,51618.0,3.0
10144,51619.0,3.0
10145,51620.0,9.0


In [10]:
nhanes78_club_1 = pd.merge(nhanes78_No_vaccination, nhanes78_Antibody, on="SEQN", how = "left")

In [11]:
nhanes78_club_2 = pd.merge(nhanes78_club_1, nhanes78_BG, on="SEQN", how = "left")

In [12]:
import pandas as pd
import pandas.io.sql as sqlio

In [13]:
pip install pandasql

Note: you may need to restart the kernel to use updated packages.


In [22]:
from pandasql import sqldf

In [23]:
query = '''
SELECT SEQN, (CASE WHEN LBXHBC = 1.0 THEN 'Positive' WHEN LBXHBC = 2.0 THEN 'Negative' WHEN LBXHBC IS NULL THEN 'Missing' END) AS Core_Antibody,
(CASE WHEN LBDHBG = 1.0 THEN 'Positive' WHEN LBDHBG = 2.0 THEN 'Negative' WHEN LBDHBG IS NULL THEN 'Missing' END) AS Surface_Antigen,
(CASE WHEN LBXHBS = 1.0 THEN 'Positive' WHEN LBXHBS = 2.0 THEN 'Negative' WHEN LBXHBS IS NULL THEN 'Missing' END) AS Surface_Antibody,
(CASE WHEN IMQ020 = 3.0 THEN 'No Doses' WHEN IMQ020 = 7.0 THEN 'Refused' WHEN IMQ020 = 9.0 THEN "Don't know" END) AS Vaccination
FROM nhanes78_club_2
'''

In [24]:
nhanes78_final = sqldf(query)

In [25]:
nhanes78_final

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination
0,41475.0,Negative,Negative,Negative,No Doses
1,41477.0,Negative,Negative,Negative,No Doses
2,41482.0,Negative,Negative,Negative,No Doses
3,41483.0,Negative,Negative,Negative,No Doses
4,41485.0,Negative,Negative,Negative,Don't know
...,...,...,...,...,...
5116,51617.0,Positive,Negative,Positive,Don't know
5117,51618.0,Negative,Negative,Negative,No Doses
5118,51619.0,Negative,Negative,Negative,No Doses
5119,51620.0,Negative,Negative,Negative,Don't know


In [26]:
query2 = '''
SELECT SEQN, Core_Antibody, Surface_Antigen, Surface_Antibody, Vaccination, 
(CASE WHEN Surface_Antigen = 'Positive' THEN 'HBV Infection' WHEN (Surface_Antibody = 'Positive') THEN 'Vaccine Immunity' 
WHEN (Surface_Antigen = 'Negative' and Core_Antibody ='Negative' and Surface_Antibody ='Negative') THEN 'No Infection'
WHEN (Core_Antibody = 'Positive' and Surface_Antibody = 'Negative') THEN 'HBV Exposure' END) AS resp_2 
FROM nhanes78_final
'''

In [27]:
nhanes78_final_resp = sqldf(query2)

In [28]:
nhanes78_final_resp[nhanes78_final_resp['resp_2'].isna()]

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2
11,41497.0,Missing,Missing,Missing,No Doses,None
16,41505.0,Missing,Missing,Missing,Don't know,None
28,41521.0,Missing,Missing,Missing,Don't know,None
34,41528.0,Missing,Missing,Missing,No Doses,None
35,41535.0,Missing,Missing,Missing,No Doses,None
...,...,...,...,...,...,...
5090,51575.0,Missing,Missing,Missing,No Doses,None
5092,51578.0,Missing,Missing,Missing,Don't know,None
5101,51591.0,Missing,Missing,Missing,No Doses,None
5106,51602.0,Missing,Missing,Missing,No Doses,None


In [29]:
nhanes78_HBV = nhanes78_final_resp[(nhanes78_final_resp['resp_2']=='HBV Infection') | (nhanes78_final_resp['resp_2']=='No Infection')]

In [32]:
# NHANES 2017-2018 Demographic Variable
nhanes78_Demographic = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/DEMO_E.XPT", format = "xport")

In [35]:
#changed columns 'RIDRETH1','DMQMILIT','DMDBORN', deleted 'DMQADFC' 
Demographic_Variables = ['SEQN','SDDSRVYR','RIAGENDR','RIDAGEYR','RIDRETH1','DMQMILIT','DMDBORN2','DMDCITZN',
                        'DMDYRSUS','DMDEDUC2','DMDMARTL','WTINT2YR','WTMEC2YR','SDMVPSU','SDMVSTRA','INDFMPIR']

In [36]:
nhanes78_Demographic = nhanes78_Demographic[Demographic_Variables]

In [37]:
nhanes78_HBV_Demo = pd.merge(nhanes78_HBV, nhanes78_Demographic, on="SEQN", how = "left")

In [38]:
nhanes78_HBV_Demo

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2,SDDSRVYR,RIAGENDR,RIDAGEYR,RIDRETH1,...,DMDBORN2,DMDCITZN,DMDYRSUS,DMDEDUC2,DMDMARTL,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDFMPIR
0,41475.0,Negative,Negative,Negative,No Doses,No Infection,5.0,2.0,62.0,5.0,...,1.0,1.0,NaN,3.0,1.0,59356.356426,60045.772497,1.0,60.0,1.83
1,41477.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,71.0,3.0,...,1.0,1.0,NaN,3.0,1.0,9935.266183,10074.150074,1.0,67.0,1.50
2,41482.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,64.0,1.0,...,2.0,2.0,7.0,2.0,1.0,9811.075078,11602.178638,2.0,65.0,4.01
3,41483.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,66.0,4.0,...,1.0,1.0,NaN,4.0,1.0,8058.685296,7920.812275,2.0,66.0,1.14
4,41485.0,Negative,Negative,Negative,Don't know,No Infection,5.0,2.0,30.0,2.0,...,4.0,1.0,4.0,2.0,1.0,19055.790060,18849.664396,2.0,71.0,1.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3710,51611.0,Negative,Negative,Negative,No Doses,No Infection,5.0,2.0,55.0,3.0,...,5.0,1.0,9.0,4.0,3.0,87778.705525,90642.023851,1.0,65.0,4.95
3711,51613.0,Negative,Negative,Negative,Don't know,No Infection,5.0,1.0,13.0,3.0,...,1.0,1.0,NaN,NaN,NaN,36824.003958,37705.152054,1.0,61.0,1.51
3712,51618.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,48.0,3.0,...,1.0,1.0,NaN,5.0,1.0,83207.549908,84181.537996,1.0,74.0,3.83
3713,51619.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,61.0,1.0,...,2.0,1.0,7.0,1.0,1.0,5197.083889,5390.204907,2.0,71.0,2.75


In [39]:
nhanes78_Insurance = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/HIQ_E.XPT", format = "xport")

In [40]:
Insurance_Variables = ['SEQN','HIQ011','HIQ031A','HIQ031B','HIQ031C','HIQ031D','HIQ031E','HIQ031H','HIQ031I','HIQ031AA']

In [41]:
nhanes78_Insurance = nhanes78_Insurance[Insurance_Variables]

In [42]:
nhanes78_HBV_Demo_Insurance = pd.merge(nhanes78_HBV_Demo, nhanes78_Insurance, on="SEQN", how = "left")

In [43]:
query4 = '''
SELECT *, 
(CASE WHEN HIQ011 = 2.0 THEN 1 ELSE 0 END) AS No_Insurance,
(CASE WHEN HIQ011 = 1.0 and HIQ031A = 14.0 THEN 1 ELSE 0 END) AS Private_Insurance,
(CASE WHEN HIQ011 = 1.0 and (HIQ031A IS NOT NULL OR HIQ031B IS NOT NULL OR HIQ031C IS NOT NULL OR HIQ031D IS NOT NULL OR 
HIQ031E IS NOT NULL OR HIQ031H IS NOT NULL OR HIQ031I IS NOT NULL OR HIQ031AA IS NOT NULL) THEN 1 ELSE 0 END) Governement_Insurance
FROM nhanes78_HBV_Demo_Insurance
'''

In [44]:
nhanes78_HBV_Demo_Insurance = sqldf(query4)

In [45]:
nhanes78_HBV_Demo_Insurance = nhanes78_HBV_Demo_Insurance.drop(columns=['HIQ011','HIQ031A','HIQ031B','HIQ031C','HIQ031D','HIQ031E','HIQ031H','HIQ031I','HIQ031AA'])

In [46]:
nhanes78_HBV_Demo_Insurance

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2,SDDSRVYR,RIAGENDR,RIDAGEYR,RIDRETH1,...,DMDEDUC2,DMDMARTL,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDFMPIR,No_Insurance,Private_Insurance,Governement_Insurance
0,41475.0,Negative,Negative,Negative,No Doses,No Infection,5.0,2.0,62.0,5.0,...,3.0,1.0,59356.356426,60045.772497,1.0,60.0,1.83,1,0,0
1,41477.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,71.0,3.0,...,3.0,1.0,9935.266183,10074.150074,1.0,67.0,1.50,0,0,1
2,41482.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,64.0,1.0,...,2.0,1.0,9811.075078,11602.178638,2.0,65.0,4.01,0,1,1
3,41483.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,66.0,4.0,...,4.0,1.0,8058.685296,7920.812275,2.0,66.0,1.14,0,1,1
4,41485.0,Negative,Negative,Negative,Don't know,No Infection,5.0,2.0,30.0,2.0,...,2.0,1.0,19055.790060,18849.664396,2.0,71.0,1.01,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3710,51611.0,Negative,Negative,Negative,No Doses,No Infection,5.0,2.0,55.0,3.0,...,4.0,3.0,87778.705525,90642.023851,1.0,65.0,4.95,0,1,1
3711,51613.0,Negative,Negative,Negative,Don't know,No Infection,5.0,1.0,13.0,3.0,...,NaN,NaN,36824.003958,37705.152054,1.0,61.0,1.51,0,1,1
3712,51618.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,48.0,3.0,...,5.0,1.0,83207.549908,84181.537996,1.0,74.0,3.83,0,1,1
3713,51619.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,61.0,1.0,...,1.0,1.0,5197.083889,5390.204907,2.0,71.0,2.75,1,0,0


In [47]:
#Alcohol use
nhanes78_Alcohol = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/ALQ_E.XPT", format = "xport")

In [48]:
nhanes78_Alcohol = nhanes78_Alcohol[['SEQN','ALQ130']]

In [49]:
nhanes78 = pd.merge(nhanes78_HBV_Demo_Insurance, nhanes78_Alcohol, on="SEQN", how = "left")

In [50]:
#Healthcare Visits
nhanes78_Healthcare = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/HUQ_E.XPT", format = "xport")

In [51]:
nhanes78_Healthcare = nhanes78_Healthcare [['SEQN','HUQ050']]

In [52]:
nhanes78 = pd.merge(nhanes78, nhanes78_Healthcare, on="SEQN", how = "left")

In [53]:
#Dialysis
nhanes78_Dialysis = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/KIQ_U_E.XPT", format = "xport")

In [54]:
nhanes787_Dialysis = nhanes78_Dialysis[['SEQN','KIQ025']]

In [55]:
nhanes78 = pd.merge(nhanes78, nhanes78_Dialysis, on="SEQN", how = "left")

In [57]:
#Liver Condition
nhanes78_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/MCQ_E.XPT", format = "xport")

In [58]:
nhanes78_Liver_Condition = nhanes78_Liver_Condition[['SEQN','MCQ160L']]

In [59]:
nhanes78 = pd.merge(nhanes78, nhanes78_Liver_Condition, on="SEQN", how = "left")

In [60]:
#Blood Transfusion
nhanes78_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/MCQ_E.XPT", format = "xport")

In [61]:
nhanes78_Blood_Transfusion = nhanes78_Blood_Transfusion[['SEQN','MCQ092']]

In [62]:
nhanes78 = pd.merge(nhanes78, nhanes78_Blood_Transfusion, on="SEQN", how = "left")

In [63]:
#Healthcare Visits time
nhanes78_Healthcare_time = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/HUQ_E.XPT", format = "xport")

In [64]:
nhanes78_Healthcare_time = nhanes78_Healthcare_time[['SEQN','HUQ060']]

In [65]:
nhanes78 = pd.merge(nhanes78, nhanes78_Healthcare_time, on="SEQN", how = "left")

In [66]:
#Hepatitis C
nhanes78_Hepatitis_C = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/HCQ_E.XPT", format = "xport")

In [67]:
nhanes78_Hepatitis_C = nhanes78_Hepatitis_C[['SEQN']]

In [68]:
import numpy as np
nhanes78_Hepatitis_C['Hepatitis_C'] = "Yes"

In [69]:
nhanes78 = pd.merge(nhanes78, nhanes78_Hepatitis_C, on="SEQN", how = "left")

In [70]:
Biochemistry_Variables = ['SEQN','LBXSATSI','LBDSALSI','LBXSASSI']

In [72]:
#Biochemistry_Variables
nhanes78_Biochemistry_Variables = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/BIOPRO_E.XPT", format = "xport")

In [73]:
nhanes78_Biochemistry_Variables = nhanes78_Biochemistry_Variables[Biochemistry_Variables]

In [74]:
nhanes78 = pd.merge(nhanes78, nhanes78_Biochemistry_Variables, on="SEQN", how = "left")

In [75]:
#Platelet count
nhanes78_Platelet_count= pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2007-2008/CBC_E.XPT", format = "xport")

In [76]:
nhanes78_Platelet_count = nhanes78_Platelet_count[['SEQN','LBXPLTSI']]

In [77]:
nhanes78 = pd.merge(nhanes78, nhanes78_Platelet_count, on="SEQN", how = "left")

In [78]:
nhanes78 = nhanes78[nhanes78['RIDAGEYR']>=18]

In [79]:
pd.set_option('display.max_columns', None)
nhanes78.head()

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2,SDDSRVYR,RIAGENDR,RIDAGEYR,RIDRETH1,DMQMILIT,DMDBORN2,DMDCITZN,DMDYRSUS,DMDEDUC2,DMDMARTL,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDFMPIR,No_Insurance,Private_Insurance,Governement_Insurance,ALQ130,HUQ050,KIQ022,KIQ025,KIQ026,KID028,KIQ005,KIQ010,KIQ042,KIQ430,KIQ044,KIQ450,KIQ046,KIQ470,KIQ050,KIQ052,KIQ480,MCQ160L,MCQ092,HUQ060,Hepatitis_C,LBXSATSI,LBDSALSI,LBXSASSI,LBXPLTSI
0,41475.0,Negative,Negative,Negative,No Doses,No Infection,5.0,2.0,62.0,5.0,2.0,1.0,1.0,NaN,3.0,1.0,59356.356426,60045.772497,1.0,60.0,1.83,1,0,0,NaN,3.000000e+00,2.0,NaN,2.0,NaN,3.0,1.0,1.0,2.0,2.0,NaN,2.0,NaN,2.0,1.0,1.0,2.0,2.0,NaN,NaN,26.0,36.0,24.0,366.0
1,41477.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,71.0,3.0,1.0,1.0,1.0,NaN,3.0,1.0,9935.266183,10074.150074,1.0,67.0,1.50,0,0,1,NaN,2.000000e+00,1.0,2.0,2.0,NaN,1.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,1.0,2.0,2.0,NaN,NaN,20.0,45.0,20.0,273.0
2,41482.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,64.0,1.0,2.0,2.0,2.0,7.0,2.0,1.0,9811.075078,11602.178638,2.0,65.0,4.01,0,1,1,4.0,1.000000e+00,2.0,NaN,2.0,NaN,5.0,3.0,2.0,NaN,1.0,4.0,2.0,NaN,3.0,3.0,2.0,1.0,2.0,NaN,NaN,42.0,45.0,35.0,239.0
3,41483.0,Negative,Negative,Negative,No Doses,No Infection,5.0,1.0,66.0,4.0,1.0,1.0,1.0,NaN,4.0,1.0,8058.685296,7920.812275,2.0,66.0,1.14,0,1,1,NaN,3.000000e+00,2.0,NaN,2.0,NaN,1.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,3.0,2.0,2.0,NaN,NaN,19.0,35.0,23.0,205.0
4,41485.0,Negative,Negative,Negative,Don't know,No Infection,5.0,2.0,30.0,2.0,2.0,4.0,1.0,4.0,2.0,1.0,19055.790060,18849.664396,2.0,71.0,1.01,1,0,0,NaN,5.397605e-79,2.0,NaN,2.0,NaN,1.0,NaN,2.0,NaN,1.0,1.0,2.0,NaN,1.0,1.0,1.0,2.0,2.0,2.0,NaN,15.0,44.0,17.0,465.0
